# Ant Simulator

I have always been fascinated by complex behavior emerging from simple rules, and ants are a perfect example of this phenomenon. 
In this example we will first discuss the basic rules that govern ant behavior, and then we will implement a simple ant simulator in Python.
Finally, we will use a C++ extension to speed up the simulation.

## Basic Rules of Ant Behavior

We have a 2D map / canvas where ants can move around. We use periodic boundary conditions, meaning that if an ant moves off one edge of the map, it reappears on the opposite edge.
On th map, there is food located at a certain position and the ants have a nest located at another position.

Ants can either be searching for food or returning to the nest with food. 
For each of these cases, the ants will drop pheromones on the map to help guide other ants.
Ie they drop two types of pheromones: one pheromone to indicate a path to food, and another pheromone to indicate a path to the nest.

When all ants start at the nest, they will look for "food pheromone" to guide them towards food. But since we just started the simulation, there is no pheromone on the map yet and the ants will move randomly.
Eventually, one or more ants will find the food and start returning to the nest, dropping "nest pheromone" on the way.
That way, a "to food" pheromone trail will be established since more ants will discover this path and follow it to the food.
The ants with food will also look for "nest pheromone" to guide them back to the nest.


Pheromones evaporate over time, meaning that if no ants walk over a certain path for a while, the pheromone concentration will decrease until it eventually disappears. Furthermore the pheromones will diffuse over time, meaning that the pheromone concentration will spread out to neighboring cells on the map.

# Python Implementation

Lets start with the class storing the state of an ant.
Here we just store the position, direction and whether the ant is carrying food or not.


In [2]:
import numpy as np
class Ant:
    def __init__(self, pos=np.array([0, 0])):
        self.pos = pos.astype(np.float32)
        self.direction = np.random.rand() * 2 * np.pi
        self.carrying_food = False

The state of the simulation is stored in the Simulation class, which contains a list of ants, the pheromone maps, and the positions of the nest and food.

In [3]:
class Simulation:
    def __init__(self, shape, n_ants):
        self.shape = shape
        self.n_ants = n_ants
        self.ants = [Ant() for _ in range(n_ants)]
        self.food_map = np.zeros((shape[0], shape[1]), dtype=np.uint8)
        self.nest_map = np.zeros((shape[0], shape[1]), dtype=np.uint8)

        # Layer 0: "To Home" trail
        # Layer 1: "To Food" trail
        self.pheromone_map = np.zeros((shape[0], shape[1], 2), dtype=np.float32)

lets implemet the function to update one step for a single ant.
In each step, an ant  can choose to turn left, right or go straight ahead based on the pheromone concentrations in front of it.

Therefore we will sample the pheromone concentrations at three positions in front of the ant (left, center, right) and use these values to compute probabilities for each of the three actions.

If the pheromone level is to low, the ant will move randomly, with a higher probability to go straight ahead.
When the pheromone levels are above a certain threshold, we will convert the pheromone levels into probabilities by normalizing them. We will add a bias st. the ant has a higher probability to go straight ahead.
Finally we will sample one of the three actions based on the computed probabilities and update the ant's position and direction accordingly.


In [4]:
def update_ant(sim, ant):
    sense_distance = 15     # Look ahead
    sense_angle = 0.5       # Radians (approx 30 deg)
    turn_angle = 0.4        # Radians


    # get rounded position and applying np.round
    rx, ry = int(round(ant.pos[0])) % sim.shape[0], int(round(ant.pos[1])) % sim.shape[1]

    # get the pheromone levels at three positions
    def get_pheromone(angle_offset):
        angle = ant.direction + angle_offset
        sense_x = int(round(ant.pos[0] + np.cos(angle) * sense_distance)) % sim.shape[0]
        sense_y = int(round(ant.pos[1] + np.sin(angle) * sense_distance)) % sim.shape[1]
        return float(sim.pheromone_map[sense_x, sense_y, int(not ant.carrying_food)])
    front = get_pheromone( 0)
    left = get_pheromone( sense_angle)
    right = get_pheromone( -sense_angle)


    if front + left + right < 0.05:
        # if all pheromone levels are very low, move randomly with bias to front
        probs = [0.8, 0.1, 0.1] 
    else:
        # convert pheromone levels to probabilities
        # add bias to front st. ants move more likely forward
        bias_front = 0.05
        front += bias_front
        total = front + left + right 
        probs = [front / total, left / total, right / total]
    
    # sample turn direction
    turn = np.random.choice(
        [0, turn_angle, -turn_angle],
        p=probs
    )

    # update direction and position
    ant.direction += turn
    ant.direction %= 2 * np.pi

    # move forward
    ant.pos[0] += np.cos(ant.direction) 
    ant.pos[1] += np.sin(ant.direction)
    ant.pos[0] %= sim.shape[0]
    ant.pos[1] %= sim.shape[1]

    # deposit pheromone
    deposit_amount = 1.0
    if ant.carrying_food:
        sim.pheromone_map[rx, ry, 1] += deposit_amount
    else:
        sim.pheromone_map[rx, ry, 0] += deposit_amount

    # check for food / home and pick up / drop food
    # and turn around
    if ant.carrying_food and sim.nest_map[rx, ry] > 0:
        ant.carrying_food = False
        ant.direction += np.pi
    elif not ant.carrying_food and sim.food_map[rx, ry] > 0:
        ant.carrying_food = True
        ant.direction += np.pi

lets implement the function to decay and diffuse the pheromone maps.
We will use a gaussian blur to simulate diffusion and just multiply the pheromone map by a decay factor to simulate evaporation.
Furthermore we clip the pheromone values to be between 0 and 50

In [5]:
from scipy.ndimage import gaussian_filter

def update_pheromone_map(sim):
    # we make the nest emit pheromones 
    home_pos = np.argwhere(sim.nest_map > 0)
    sim.pheromone_map[home_pos[:,0], home_pos[:,1], 0] += 1.0

    # we make the food emit pheromones (actually the food would just emit food scent, but for simplicity we use the same map)
    food_pos = np.argwhere(sim.food_map > 0)
    sim.pheromone_map[food_pos[:,0], food_pos[:,1], 1] += 1.0
    
    # diffuse the pheromones
    for i in range(2):
        sim.pheromone_map[:, :, i] = gaussian_filter(sim.pheromone_map[:, :, i], sigma=0.3)

    # decay
    sim.pheromone_map *= 0.99

    # cap the pheromone values
    sim.pheromone_map = np.clip(sim.pheromone_map, 0, 50)


Since we want to view the simulation, we will implement a drawing function using ipycanvas.

In [6]:
from ipycanvas import Canvas, hold_canvas

def draw(sim, canvas):
    with hold_canvas(canvas):
        img = np.zeros((sim.shape[0], sim.shape[1], 3), dtype=np.uint8)

        p0 = np.clip(sim.pheromone_map[:, :, 0] * 50, 0, 255)
        p1 = np.clip(sim.pheromone_map[:, :, 1] * 50, 0, 255)
        
        img[:, :, 1] = p0 # Gree TO HOME path
        img[:, :, 0] = p1 # red  TO FOOD path
        
        img[sim.food_map > 0] = [255, 0, 0] # red food
        img[sim.nest_map > 0] = [255, 255, 255] # white nest

        # swap 0,1 axis for canvas
        img = np.transpose(img, (1, 0, 2))
        canvas.put_image_data(img, 0, 0)

        # draw ants
        for ant in sim.ants:
            x, y = int(ant.pos[0]), int(ant.pos[1])
            color = (0, 0, 255)
            canvas.fill_style = f'rgb{color}'
            canvas.fill_rect(x, y, 2, 2)
    

Putting it all together, we can now run the simulation and visualize the results.
We will run the simulation for a fixed time (20 sec by default)

In [14]:
import time
from IPython.display import display
from ipycanvas import Canvas, hold_canvas

n_ants = 1000
shape = (300, 300)
sim = Simulation(shape, n_ants)
canvas = Canvas(width=shape[0], height=shape[1])
display(canvas)

# Circular blobs for nest and food
cx, cy = 100, 100
y, x = np.ogrid[-10:10, -10:10]
mask = x**2 + y**2 <= 10**2
sim.food_map[cx-10:cx+10, cy-10:cy+10][mask] = 1
cx, cy = 200, 200
sim.nest_map[cx-10:cx+10, cy-10:cy+10][mask] = 1


# initialize ants at the nest
nest_positions = np.argwhere(sim.nest_map > 0)
for ant in sim.ants:
    idx = np.random.randint(len(nest_positions))
    ant.pos = nest_positions[idx].astype(np.float32)


t0 = time.time()
while True:
    for i in range(5): # update multiple times per draw
        for ant in sim.ants:
            update_ant(sim, ant)
        update_pheromone_map(sim)

    draw(sim, canvas)
    # quit after 20 seconds
    if time.time() - t0 > 20:
        break


Canvas(height=300, width=300)

# C++ Extension

Since the simulation can be quite slow in Python, especially with a large number of ants, we implemented a C++ extension to speed up the simulation.
The C++ extension implements the same logic as the Python version, but is much faster due to the compiled nature of C++.
The simulation will run for a fixed time (15 sec)

In [15]:
from pathlib import Path
import sys
import dtks
import numpy as np
import ipycanvas
import time

shape = [600,600]
params = dtks.Parameters()
params.shape = shape                           # width, height of the simulation
params.pheromone_evaporation_rate = 0.003      # how fast pheromones evaporate, between 0 and 1
params.sigma_diffusion = 0.4                   # diffusion sigma for pheromones, bigger = more diffusion
params.sense_distance = 20                     # how far away ants can sense pheromones
params.sense_angle = 0.5                       # how much do angle look to the left/right when sensing pheromones
params.turn_angle = 0.4                        # how much do ants turn when changing direction
params.beta_uniformity = 0.1                   # a bias to favor going straight. A bigger value means ants walk more likely straight
params.infinite_food = True                    # when ants find food, will it disappear or not
params.seed = 42                               # random seed
params.nest_pheromone_deposit_amount = 100.0   # amount of pheromone the nest (and the food) emit every step
params.pheromone_deposit_amount = 1.0          # how much pheromone an ant deposits every step

sim = dtks.AntSimulation(params)

# make nest map
cx, cy = shape[0] // 2, shape[1] // 2
nest_map = sim.nest_map()
nest_map[cx-10:cx+10, cy-0:cy+20] = 1

# make food map
food_map = sim.food_map()
food_map[cx-200:cx-180, cy-200:cy-180] = 50  

# mark the simulation as ready to start
sim.ready()

canvas = ipycanvas.Canvas(width=shape[1], height=shape[0])
display(canvas)
draw_image = np.zeros((shape[0], shape[1], 4), dtype=np.uint8)
t0 = time.time()
while True:
    # run 10 steps every frame
    for i in range(10):
        sim.step()
    sim.draw(draw_image)  
    canvas.put_image_data(draw_image, 0, 0)
    if time.time() - t0 > 15:
        break

Canvas(height=600, width=600)

# Adding Walls

To make the simulation more interesting, we can add walls to the map that the ants cannot cross.

In [ ]:
from pathlib import Path
import sys
import dtks
import numpy as np
import ipycanvas
import time

shape = [600,600]
params = dtks.Parameters()
params.shape = shape
params.pheromone_evaporation_rate = 0.003
params.sigma_diffusion = 0.4
params.sense_distance = 20
params.sense_angle = 0.5
params.turn_angle = 0.4
params.beta_uniformity = 0.1
params.infinite_food = True
params.seed = 42
params.nest_pheromone_deposit_amount = 100.0
params.pheromone_deposit_amount = 1.0

def circular_mask(shape, radius, center=None):
    h, w = shape
    if center is None:
        center = (h // 2, w // 2)

    Y, X = np.ogrid[:h, :w]
    dist = (X - center[1])**2 + (Y - center[0])**2

    return dist <= radius**2


params.nest_pheromone_deposit_amount = 10.0
params.beta_uniformity = 0.1
params.sense_distance = 20
sim = dtks.AntSimulation(params)




# # make land mask
# make land mask with border
is_land = sim.is_land()
cx, cy = shape[0] // 2, shape[1] // 2
# is_land = circular_mask(shape, radius=1500/2, center=(cx, cy)).astype(np.uint8)
is_land[:20, :] = 0
is_land[-20:, :] = 0
is_land[:, :20] = 0
is_land[:, -20:] = 0

# add a few strategic barriers to make it challenging but solvable
is_land[200:220, 150:450] = 0  # horizontal barrier
is_land[350:370, 200:500] = 0  # lower horizontal barrier
is_land[150:350, 280:300] = 0  # center vertical barrier
is_land[100:120, 400:550] = 0  # upper right barrier
is_land[450:470, 100:350] = 0  # lower left barrier
is_land[250:270, 450:550] = 0  # right side barrier
is_land[50:250, 450:470] = 0   # upper right vertical barrier


# make nest map
nest_map = sim.nest_map()
nest_map[cx-10:cx+10, cy-0:cy+20] = 1

# make food map
food_map = sim.food_map()
food_map[cx-200:cx-180, cy-200:cy-180] = 50  

sim.ready()

canvas = ipycanvas.Canvas(width=shape[1], height=shape[0])
display(canvas)
draw_image = np.zeros((shape[0], shape[1], 4), dtype=np.uint8)
t0 = time.time()
while True:
    for i in range(10):
        sim.step()
    sim.draw(draw_image)  
    canvas.put_image_data(draw_image, 0, 0)
    if time.time() - t0 > 20:
        break

# Chaging the environment

To make the simulation more dynamic, we can change the location of the food every 200 steps. This will force the ants to adapt and find new paths to the food.

In [16]:
from pathlib import Path
import sys
import dtks
import numpy as np
import ipycanvas

shape = [600,600]
params = dtks.Parameters()
params.shape = shape
params.pheromone_evaporation_rate = 0.003
params.sigma_diffusion = 0.4
params.sense_distance = 20
params.sense_angle = 0.5
params.turn_angle = 0.4
params.beta_uniformity = 0.1
params.infinite_food = True
params.seed = 42
params.nest_pheromone_deposit_amount = 10.0
params.pheromone_deposit_amount = 1.0

def circular_mask(shape, radius, center=None):
    h, w = shape
    if center is None:
        center = (h // 2, w // 2)

    Y, X = np.ogrid[:h, :w]
    dist = (X - center[1])**2 + (Y - center[0])**2

    return dist <= radius**2

params.nest_pheromone_deposit_amount = 10.0
params.beta_uniformity = 0.1
params.sense_distance = 20
sim = dtks.AntSimulation(params)


# # make land mask
# make land mask with border
is_land = sim.is_land()
cx, cy = shape[0] // 2, shape[1] // 2
is_land[:20, :] = 0
is_land[-20:, :] = 0
is_land[:, :20] = 0
is_land[:, -20:] = 0


# make nest map
nest_map = sim.nest_map()
nest_map[cx-10:cx+10, cy-0:cy+20] = 1


# make food map
food_map = sim.food_map()
food_map[cx-200:cx-180, cy-200:cy-180] = 1  

sim.ready()

canvas = ipycanvas.Canvas(width=shape[1], height=shape[0])
display(canvas)
draw_image = np.zeros((shape[0], shape[1], 4), dtype=np.uint8)
t0 = time.time()
step = 0
while True:

    # change location of food every 100 steps
    if step % 200 == 0: 
        food_map[:] = 0
        fx = np.random.randint(50, shape[0]-50)
        fy = np.random.randint(50, shape[1]-50)
        food_map[fx-10:fx+10, fy-10:fy+10] = 255


    for i in range(10):
        sim.step()
    sim.draw(draw_image)  
    canvas.put_image_data(draw_image, 0, 0)
    if time.time() - t0 > 20:
        break
    step += 1

Canvas(height=600, width=600)

# Make food disappear after it has been collected

when setting `params.infinite_food = False`, the food map count will decrease as ants collect food.
When (allmost) all food has been collected, we will place new food at a random location on the map.

In [ ]:
from pathlib import Path
import sys
import dtks
import numpy as np
import ipycanvas

shape = [600,600]
params = dtks.Parameters()
params.shape = shape
params.pheromone_evaporation_rate = 0.003
params.sigma_diffusion = 0.4
params.sense_distance = 20
params.sense_angle = 0.5
params.turn_angle = 0.4
params.beta_uniformity = 0.1
params.infinite_food = False
params.seed = 42
params.nest_pheromone_deposit_amount = 1.0
params.pheromone_deposit_amount = 1.0

def circular_mask(shape, radius, center=None):
    h, w = shape
    if center is None:
        center = (h // 2, w // 2)

    Y, X = np.ogrid[:h, :w]
    dist = (X - center[1])**2 + (Y - center[0])**2

    return dist <= radius**2


params.nest_pheromone_deposit_amount = 10.0
params.beta_uniformity = 0.1
params.sense_distance = 20
sim = dtks.AntSimulation(params)


# # make land mask
# make land mask with border
is_land = sim.is_land()
cx, cy = shape[0] // 2, shape[1] // 2
is_land[:20, :] = 0
is_land[-20:, :] = 0
is_land[:, :20] = 0
is_land[:, -20:] = 0


# make nest map
nest_map = sim.nest_map()
nest_map[cx-10:cx+10, cy-0:cy+20] = 1

# make food map
food_map = sim.food_map()
food_map[cx-200:cx-180, cy-200:cy-180] = 1  # the amount of food in this cell 

sim.ready()

canvas = ipycanvas.Canvas(width=shape[1], height=shape[0])
display(canvas)
draw_image = np.zeros((shape[0], shape[1], 4), dtype=np.uint8)
t0 = time.time()
while True:
    # change location of food when less then 20 units of food are left
    if food_map.sum() <= 40:
        fx = np.random.randint(50, shape[0]-50)
        fy = np.random.randint(50, shape[1]-50)
        food_map[:] = 0
        food_map[fx-10:fx+10, fy-10:fy+10] = 1


    for i in range(10):
        sim.step()
    sim.draw(draw_image)  
    canvas.put_image_data(draw_image, 0, 0)
    if time.time() - t0 > 40:
        break


Canvas(height=600, width=600)